In [10]:
print("ok")

ok


In [11]:
import os

In [12]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2


In [13]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [14]:
key = os.getenv("OPENAI_API_KEY")

In [15]:
llmobj=ChatOpenAI(openai_api_key = key, model = "gpt-3.5-turbo", temperature=0.7)

/Users/mayankchugh/gitRepos/mayankchugh.learning/MCQ-Generator/mcqvenv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [16]:
%pwd

'/Users/mayankchugh/gitRepos/mayankchugh.learning/MCQ-Generator/experiment'

In [17]:
projectPath = os.getenv("PATH_TO_PROJECT")
projectPath

'/Users/mayankchugh/gitRepos/mayankchugh.learning/MCQ-Generator/'

In [18]:
mcqPrjPath = os.getenv("MCQ_PROJECT_FOLDER")
mcqPrjPath

'MCQ-Generator/'

In [19]:
with open(projectPath+"Response.json","r") as f:
    RESPONSE_JSON=json.load(f)

In [20]:
print(RESPONSE_JSON)

{'1': {'no': '1', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'no': '2', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'no': '3', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [21]:
TEMPLATE="""Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}

"""

In [22]:
quiz_generation_prompt = PromptTemplate (
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    template=TEMPLATE
)

In [23]:
quiz_chain=LLMChain(llm=llmobj,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [24]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [25]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [26]:
review_chain = LLMChain(llm=llmobj,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [27]:
generate_evaluate_chain = SequentialChain(
    chains= [quiz_chain,review_chain], 
    input_variables=["text","number","subject","tone","RESPONSE_JSON"], 
    output_variables=["quiz", "review"], 
    verbose=True)

In [28]:
DATA_PATH=projectPath+"data.txt"
DATA_PATH

'/Users/mayankchugh/gitRepos/mayankchugh.learning/MCQ-Generator/data.txt'

In [29]:
with open(DATA_PATH,"r") as file:
    TEXT=file.read()

In [30]:
print(TEXT)

Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:

Systems, architecture and infrastructure for Generative AI
Machine learning and Modeling using LLMs and Diffusion models
Large Language Models and its applications
Multi-modal Generative AI and its applications
Gen AI based Plugins and

TEXT

NUMBER=5

SUBJECT="AI"

TONE="Simple"  

RESPONSE_JSON=RESPONSE_JSON

In [35]:
TEXT
NUMBER=5
SUBJECT="AI"
TONE="Simple",
RESPONSE_JSON=RESPONSE_JSON

In [36]:
RESPONSE_JSON

{'1': {'no': '1',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'no': '2',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'no': '3',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [37]:
import json
json.dumps(RESPONSE_JSON)

'{"1": {"no": "1", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"no": "2", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"no": "3", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [38]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "RESPONSE_JSON":json.dumps(RESPONSE_JSON)
        }
    )

/Users/mayankchugh/gitRepos/mayankchugh.learning/MCQ-Generator/mcqvenv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Text:Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:

Systems, architecture and infrastructure for Generative AI
Machine learning and Modeling using LLMs and Diffusion models
Large L

In [39]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1397
Prompt Tokens:965
Completion Tokens:432
Total Cost:0.0023115


In [40]:
response

{'text': 'Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:\n\nSystems, architecture and infrastructure for Generative AI\nMachine learning and Modeling using LLMs and Diffusion models\nLarge Language Models and its applications\nMulti-modal Generative AI and its applications\nGen AI b

In [41]:
quiz=response.get("quiz")

In [42]:
quiz=json.loads(quiz)

In [43]:
quiz

{'1': {'no': '1',
  'mcq': 'What is the focus of the workshop on Generative AI at AIMLSystems?',
  'options': {'a': 'Building robots',
   'b': 'Creating realistic text, images, and video',
   'c': 'Developing self-driving cars',
   'd': 'Studying ancient civilizations'},
  'correct': 'b'},
 '2': {'no': '2',
  'mcq': 'What are some topics of interest for the workshop on Generative AI?',
  'options': {'a': 'Cooking recipes',
   'b': 'Systems, architecture, and infrastructure for Generative AI',
   'c': 'Fishing techniques',
   'd': 'Fashion design trends'},
  'correct': 'b'},
 '3': {'no': '3',
  'mcq': 'Which models are used in Machine learning and Modeling for Generative AI?',
  'options': {'a': 'RNNs and CNNs',
   'b': 'LLMs and Diffusion models',
   'c': 'Decision trees',
   'd': 'K-means clustering'},
  'correct': 'b'},
 '4': {'no': '4',
  'mcq': 'What is one of the applications of Large Language Models mentioned in the text?',
  'options': {'a': 'Building space shuttles',
   'b': 'D

In [44]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [45]:
quiz_table_data

[{'MCQ': 'What is the focus of the workshop on Generative AI at AIMLSystems?',
  'Choices': 'a: Building robots | b: Creating realistic text, images, and video | c: Developing self-driving cars | d: Studying ancient civilizations',
  'Correct': 'b'},
 {'MCQ': 'What are some topics of interest for the workshop on Generative AI?',
  'Choices': 'a: Cooking recipes | b: Systems, architecture, and infrastructure for Generative AI | c: Fishing techniques | d: Fashion design trends',
  'Correct': 'b'},
 {'MCQ': 'Which models are used in Machine learning and Modeling for Generative AI?',
  'Choices': 'a: RNNs and CNNs | b: LLMs and Diffusion models | c: Decision trees | d: K-means clustering',
  'Correct': 'b'},
 {'MCQ': 'What is one of the applications of Large Language Models mentioned in the text?',
  'Choices': 'a: Building space shuttles | b: Developing social media platforms | c: Creating realistic text | d: Growing crops',
  'Correct': 'c'},
 {'MCQ': 'What is emphasized in the text rega

In [46]:
df=pd.DataFrame(quiz_table_data)

In [47]:
df.to_csv("genai.csv",index=False)

In [48]:
LOG_FILE="{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

In [49]:
LOG_FILE

"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

In [50]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'03_28_2024_17_33_45'